## Regresison models for 

### model ①

metric_level gained =  function(FLcumulativecompetence, interact_classmates, metric_Gender, motivationdegree, voterank, out_degree_weighted, degree_weighted, betweenness_weighted)

### model ② 

metric_level gained =  function(FLcumulativecompetence, interact_classmates, metric_Gender, motivationdegree + tylko te miary centralności, których kontrybucja w powyższym jest istotna (przedział ufności nie zawiera 0))

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

## Load data

In [ ]:
network_type = "tl123_edges"
df_merged = pd.DataFrame()


for i in range(1, 4, 1):
    eda_path = Path(f"analysis/{network_type}/{i}_eda.csv")
    eda = pd.read_csv(eda_path, index_col=0)
    print(len(eda), len(eda.columns))
    df_merged = pd.concat([df_merged, eda], ignore_index=True)

print(len(df_merged), len(df_merged.columns))

In [ ]:
df_merged.head()

## Select columns to be used in regression

In [ ]:
reg_df = df_merged[
    [
        "metric_level gained",
        "metric_FLcumulativecompetence",
        "interact_classmates",
        "metric_Gender",
        "psycho_motivationdegree",

        #     "betweenness_weighted",
        # "betweenness",
        # "in_degree_weighted",
        # "in_degree",
        # "out_degree_weighted",
        # "out_degree",
        # "degree_weighted",
        # "degree",
        "pagerank_weighhted",
        # 'pagerank',
        "closeness",
        # "voterank",
        #    "clustering_coefficient",
    ]
]

reg_df.head()

## Clean up data (convert strings to floats, fill NaNs)

In [ ]:
cols_with_nans = []
for col in reg_df.columns:
    if reg_df[col].isnull().values.any():
        print(col, reg_df[col].unique())
        cols_with_nans.append(col)

In [ ]:
reg_df = reg_df.fillna({col: 0 for col in cols_with_nans})
reg_df.dtypes

In [ ]:
reg_df["metric_Gender"].unique()

In [ ]:
reg_df["gender"] = reg_df["metric_Gender"].apply(lambda x: 0. if x == "female" else 1.)
reg_df["metric_Gender"] = reg_df["gender"]
reg_df = reg_df.drop(["gender"], axis=1)

In [ ]:
reg_df.hist()
plt.tight_layout()
plt.show()

In [ ]:
nreg_df=(reg_df-reg_df.min())/(reg_df.max()-reg_df.min())

In [ ]:
nreg_df.hist()
plt.tight_layout()
plt.show()

## Perform regression

In [ ]:
# https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
# https://medium.com/swlh/interpreting-linear-regression-through-statsmodels-summary-4796d359035a

X = nreg_df.drop(["metric_level gained"], axis=1)
y = nreg_df["metric_level gained"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())


In [ ]:
with open("regression_with_interception.csv", "w") as file:
    file.write(est2.summary().as_csv())